In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *
from fastai.vision.widgets import *
from fastai.callback.tensorboard import *

IN_NOTEBOOK = False

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
import datetime
import pandas as pd
import numpy as np

In [5]:
path = Path('../data/train')

In [6]:
path.ls()

(#3) [Path('../data/train/healthy_wheat'),Path('../data/train/leaf_rust'),Path('../data/train/stem_rust')]

In [7]:
def sort_image_by_dimensions(path, size):
    w, h = Image.open(path).size
    return w >= size and h >= size

In [8]:
image_size = 460

In [9]:
def correct_sized_images(path):
    return [img for img in get_image_files(path) if sort_image_by_dimensions(img, image_size)]

In [10]:
def get_y(r): return parent_label(r).split(" ")

In [11]:
kwargs = {'num_workers': 1, 'pin_memory': True}

def get_dls(bs, size):
    dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                        splitter=RandomSplitter(seed=42),
                        get_items=correct_sized_images,
                        get_y=get_y,
                        item_tfms=Resize(image_size),
                        batch_tfms=aug_transforms(size=size, min_scale=0.75))

    return dblock.dataloaders(path, bs=bs, num_workers=0).cuda()

In [12]:
batch_size = 64
dls = get_dls(batch_size, 64)

In [13]:
# dls.show_batch(max_n=4, nrows=1)

In [14]:
cuda0 = torch.device('cuda:0')
cpu = torch.device('cpu')

In [15]:
from fastai.losses import *
from torch.nn.modules import loss

class smooth_binary_cross_entropy(loss._Loss):

    def __init__(self, weight: Optional[Tensor] = None, size_average=None, reduce=None, reduction: str = 'mean',
             pos_weight: Optional[Tensor] = None) -> None:
        super(smooth_binary_cross_entropy, self).__init__(size_average, reduce, reduction)
        self.register_buffer('weight', weight)
        self.register_buffer('pos_weight', pos_weight)

    def forward(self, input: Tensor, target: Tensor) -> Tensor:
        c = target.shape[1]
        eps = 0.1
        smoothed_target = torch.where(target==1, 1-(eps+(eps/c)), eps/c)
        return F.binary_cross_entropy_with_logits(input,
                                                  smoothed_target,
                                                  pos_weight=self.pos_weight,
                                                  reduction=self.reduction)

@delegates()
class smooth_loss_v2(BaseLoss):
    "Same as `nn.BCEWithLogitsLoss`, but flattens input and target."
    @use_kwargs_dict(keep=True, weight=None, reduction='mean', pos_weight=None)
    def __init__(self, *args, axis=-1, floatify=True, thresh=0.5, **kwargs):
        if kwargs.get('pos_weight', None) is not None and kwargs.get('flatten', None) is True:
            raise ValueError("`flatten` must be False when using `pos_weight` to avoid a RuntimeError due to shape mismatch")
        if kwargs.get('pos_weight', None) is not None: kwargs['flatten'] = False
        super().__init__(smooth_binary_cross_entropy, *args, axis=axis, floatify=floatify, is_2d=False, **kwargs)
        self.thresh = thresh

    def decodes(self, x):    return x>self.thresh
    def activation(self, x): return torch.sigmoid(x)

In [16]:
pos_weight = torch.ones(3, device=cuda0)

In [17]:
learn = cnn_learner(dls,
                resnet50,
                metrics=partial(accuracy_multi, thresh=0.2),
                loss_func=smooth_loss_v2(pos_weight=pos_weight),
                cbs=[MixUp(0.1), CudaCallback])

In [18]:
#learn.lr_find()

In [19]:
class FineTuner():

    def __init__(self, learner: Learner):
        self.learner = learner

    def fit_with_presizing(self,
                           presizing_cycles: int,
                           max_image_size: int,
                           image_sizes: Optional[list] = None,
                           epochs_per_cycle: Optional[list] = None,
                           batch_size: int = 64):

        if isinstance(image_sizes, type(None)):
            image_sizes = [int(max_image_size/(presizing_cycles - i)) for i in range(presizing_cycles)]

        if isinstance(epochs_per_cycle, type(None)):
            epochs_per_cycle = [5]

        for i in range(presizing_cycles):
            image_size_cycle = self.get_first_index_else_last(image_sizes, i, presizing_cycles)
            epochs = self.get_first_index_else_last(epochs_per_cycle, i, presizing_cycles)

            print(f"Fine tuning cycle '{i + 1}' for {epochs} epochs with parameters\n"
                  f"\tbatch_size: {batch_size},\n"
                  f"\timage_size: {image_size_cycle}:")

            self.learner.dls = get_dls(batch_size, image_size_cycle)
            self.fine_tune_with_epoch(epochs)
            print("")

    def fine_tune_with_epoch(self, epochs: int):
        with self.learner.no_bar():
            self.learner.fine_tune(epochs)

    def get_first_index_else_last(self, list_obj: list, idx: int, total_iters: int):
        item = list_obj[0]

        if idx == total_iters - 1:
            item = list_obj[-1]

        return list_obj[idx] if idx < len(list_obj) - 1 else item

In [20]:
tuner = FineTuner(learn)
tuner.fit_with_presizing(presizing_cycles=5,
                         epochs_per_cycle=[5, 10],
                         max_image_size=460)

Fine tuning cycle '1' for 5 epochs with parameters
	batch_size: 64,
	image_size: 92:
[0, 1.0562528371810913, 0.897773027420044, 0.3549382984638214, '01:21']


KeyboardInterrupt: 

In [ ]:
# with learn.no_bar():
#     learn.fine_tune(5, freeze_epochs=3)

In [ ]:
# learn.dls = get_dls(batch_size, 128)

In [ ]:
# with learn.no_bar():
#     learn.fine_tune(5)

In [ ]:
# learn.dls = get_dls(batch_size, 256)

In [ ]:
# with learn.no_bar():
#     learn.fine_tune(10)

In [ ]:
learn.loss_func

In [ ]:
learn.recorder.plot_loss()

In [ ]:
x,y = dls.one_batch()

In [ ]:
import random as rnd

preds,t = learn.get_preds(dl=[(x,y)])
rand_idx = rnd.randint(0, len(preds))-1
print(preds[rand_idx], t[rand_idx])

In [ ]:
preds[0].sum()

In [ ]:
learn.model(x).shape

In [ ]:
model_name = "resnet18"
time_now = datetime.datetime.now().strftime(format='%H%M%S')

export_path = Path('../models/exported')

model_filename = export_path/f"{time_now}_{model_name}_B{batch_size}S{image_size}.pkl"
learn.export(fname=model_filename)

In [ ]:
learn.dls.vocab